In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 64
p1 = .5

class ResBlock(nn.Module):
    def __init__(self, in_size, out_size, kernel_size, stride, padding, padding_mode, encoder):
        super().__init__()
        if (encoder):
            self.conv1 = nn.Conv2d(in_size, in_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
            self.conv2 = nn.Conv2d(in_size, out_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
        else:
            self.conv1 = nn.ConvTranspose2d(in_size, in_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
            self.conv2 = nn.ConvTranspose2d(in_size, out_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(in_size)
        self.batchnorm2 = nn.BatchNorm2d(out_size)
        
        self.dropout = nn.Dropout(p=p1)        

    def convblock(self, x):
        x = self.dropout(F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1))
        x = self.dropout(F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1))
        x = self.dropout(F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1))
        x = self.dropout(F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1))
        return x
   
    def forward(self, x): 
        y = self.dropout(F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1))
        return y + self.convblock(x)
    
    
    
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.resblock1 = ResBlock(in_size=1, out_size=channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode, encoder=True)
        self.resblock2 = ResBlock(in_size=channels_base, out_size=channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode, encoder=True)
        
        self.resblock3 = ResBlock(in_size=channels_base * 2, out_size=channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode, encoder=False) 
        self.resblock4 = ResBlock(in_size=channels_base, out_size=channels_base //2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode, encoder=False) 
        self.convtrans1 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm2 = nn.BatchNorm2d(channels_base)
        
        self.dropout = nn.Dropout(p=p1)        
        
    def forward(self, input):
        # encoder
        
        x = self.resblock1(input)
        y1 = self.dropout(F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1))
        x = F.avg_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = self.resblock2(x)
        y2 = self.dropout(F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1))
        x = F.avg_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = F.interpolate(self.resblock3(x), scale_factor=scale_factor)
        x = F.interpolate(self.resblock4(x + y2), scale_factor=scale_factor)
        x = self.convtrans1(x + y1)


        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]             640
       BatchNorm2d-2           [-1, 64, 32, 32]             128
           Dropout-3           [-1, 64, 32, 32]               0
            Conv2d-4            [-1, 1, 32, 32]              10
       BatchNorm2d-5            [-1, 1, 32, 32]               2
           Dropout-6            [-1, 1, 32, 32]               0
            Conv2d-7            [-1, 1, 32, 32]              10
       BatchNorm2d-8            [-1, 1, 32, 32]               2
           Dropout-9            [-1, 1, 32, 32]               0
           Conv2d-10            [-1, 1, 32, 32]              10
      BatchNorm2d-11            [-1, 1, 32, 32]               2
          Dropout-12            [-1, 1, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]             640
      BatchNorm2d-14           [-1, 64,

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.04158309 (0.61624984), PSNR 13.81083107 (9.52920276), SSIM 0.30656821 (0.12147072)
Finished training epoch 0


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 75 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 3 pixels
  return func(*args, **kwargs)


Validate: MSE 0.00626741 (0.00503957), PSNR 22.02911758 (23.01596959), SSIM 0.55355436 (0.64313419)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.02174655 (0.03066210), PSNR 16.62609673 (15.20695793), SSIM 0.48114499 (0.40439570)
Finished training epoch 1
Validate: MSE 0.00416673 (0.00333391), PSNR 23.80204391 (24.84590424), SSIM 0.66451317 (0.74320890)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.01373496 (0.01774704), PSNR 18.62172318 (17.55231757), SSIM 0.57996130 (0.52440570)
Finished training epoch 2
Validate: MSE 0.00353870 (0.00290737), PSNR 24.51156425 (25.43656108), SSIM 0.69088936 (0.76502161)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00970962 (0.01146221), PSNR 20.12797928 (19.43774512), SSIM 0.59685850 (0.58997042)
Finished training epoch 3
Validate: MSE 0.00333046 (0.00280155), PSNR 24.77496147 (25.59732602), SSIM 0.69739425 (0.76942242)
Finished validation.
Starting training epoch 4
Epoch: 4, MSE 0.00673049 (0.008018

Validate: MSE 0.00365400 (0.00294418), PSNR 24.37230873 (25.39011725), SSIM 0.70551813 (0.77863922)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00218023 (0.00246125), PSNR 26.61498070 (26.11317756), SSIM 0.77400458 (0.76754036)
Finished training epoch 31
Validate: MSE 0.00320082 (0.00272188), PSNR 24.94738960 (25.72151927), SSIM 0.70606959 (0.77827184)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00246654 (0.00244757), PSNR 26.07912254 (26.13611502), SSIM 0.77081430 (0.76856567)
Finished training epoch 32
Validate: MSE 0.00330121 (0.00275505), PSNR 24.81326485 (25.67078484), SSIM 0.70635027 (0.77796502)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00256081 (0.00242950), PSNR 25.91621971 (26.16638973), SSIM 0.76328093 (0.77014523)
Finished training epoch 33
Validate: MSE 0.00348684 (0.00284924), PSNR 24.57568169 (25.53371076), SSIM 0.70725000 (0.77966498)
Finished validation.
Starting training epoch 34
Epoch: 34, MSE 0.0029362

Validate: MSE 0.00330733 (0.00278689), PSNR 24.80522728 (25.62406715), SSIM 0.70546454 (0.77669221)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00230377 (0.00232561), PSNR 26.37561607 (26.35876328), SSIM 0.78107101 (0.77564136)
Finished training epoch 61
Validate: MSE 0.00328693 (0.00274796), PSNR 24.83209419 (25.68769883), SSIM 0.70442426 (0.77596937)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00265977 (0.00232086), PSNR 25.75156212 (26.36838937), SSIM 0.76950371 (0.77575361)
Finished training epoch 62
Validate: MSE 0.00317721 (0.00293459), PSNR 24.97954369 (25.42405443), SSIM 0.70701742 (0.77657142)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00216534 (0.00233151), PSNR 26.64472961 (26.34671332), SSIM 0.76652461 (0.77513164)
Finished training epoch 63
Validate: MSE 0.00340515 (0.00283156), PSNR 24.67864037 (25.54827503), SSIM 0.70577902 (0.77673424)
Finished validation.
Starting training epoch 64
Epoch: 64, MSE 0.0019535

Validate: MSE 0.00333769 (0.00279041), PSNR 24.76554680 (25.61522031), SSIM 0.70540869 (0.77633886)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00209803 (0.00226437), PSNR 26.78188515 (26.47152821), SSIM 0.77518129 (0.77655838)
Finished training epoch 91
Validate: MSE 0.00310120 (0.00276244), PSNR 25.08470345 (25.66837859), SSIM 0.70566821 (0.77689146)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00211495 (0.00226249), PSNR 26.74700737 (26.47604463), SSIM 0.77932155 (0.77671242)
Finished training epoch 92
Validate: MSE 0.00340156 (0.00329561), PSNR 24.68321609 (24.95425591), SSIM 0.70674467 (0.77571976)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00218130 (0.00226881), PSNR 26.61284065 (26.46578469), SSIM 0.75472063 (0.77649119)
Finished training epoch 93
Validate: MSE 0.00318353 (0.00283286), PSNR 24.97090912 (25.56627350), SSIM 0.70595765 (0.77601314)
Finished validation.
Starting training epoch 94
Epoch: 94, MSE 0.0021096

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00338306 (0.00294666), PSNR 24.70690346 (25.39151273), SSIM 0.70477033 (0.77522550)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()